### This file is for testing radical changes to the codebase. Upside Down Reinforcement Learning.py is the working version.

In [17]:
import time
from copy import deepcopy
import gym
import numpy as np
import torch
import torch.nn.functional as F

In [18]:
env = gym.make('LunarLander-v2')

In [19]:
#command takes form [derired reward, desired horizon]
def random_policy(obs, command):
    return np.random.randint(env.action_space.n)

In [20]:
#Visualise agent function
def visualise_agent(policy, command, n=5):
    try:
        for trial_i in range(n):
            current_command = deepcopy(command)
            observation = env.reset()
            done=False
            t=0
            episode_return=0
            while not done:
                env.render()
                action = policy(torch.tensor([observation]).double(), torch.tensor([command]).double())
                observation, reward, done, info = env.step(action)
                episode_return+=reward
                current_command[0]-= reward
                current_command[1] = max(1, current_command[1]-1)
                t+=1
            env.render()
            time.sleep(1.5)
            print("Episode {} finished after {} timesteps. Return = {}".format(trial_i, t, episode_return))
        env.close()
    except KeyboardInterrupt:
        env.close()

In [21]:
#visualise_agent(random_policy, command=[500, 500], n=3)

In [22]:
class FCNN_AGENT(torch.nn.Module):
    def __init__(self, command_scale):
        super().__init__()
        embedding_size=32
        hidden_size=64
        self.command_scale=command_scale
        self.observation_embedding = torch.nn.Sequential(
            torch.nn.Linear(np.prod(env.observation_space.shape), embedding_size),
            torch.nn.Tanh()
        )
        self.command_embedding = torch.nn.Sequential(
            torch.nn.Linear(2, embedding_size),
            torch.nn.Sigmoid()
        )
        self.to_output = torch.nn.Sequential(
            torch.nn.Linear(embedding_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, env.action_space.n)
        )
    
    def forward(self, observation, command):
        obs_emebdding = self.observation_embedding(observation)
        cmd_embedding = self.command_embedding(command*self.command_scale)
        embedding = torch.mul(obs_emebdding, cmd_embedding)
        action_prob_logits = self.to_output(embedding)
        return action_prob_logits
    
    def create_optimizer(self, lr):
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

In [23]:
def collect_experience(policy, replay_buffer, replay_size, last_few, n_episodes=100, log_to_tensorboard=True):
    global i_episode
    init_replay_buffer = deepcopy(replay_buffer)
    try:
        for _ in range(n_episodes):
            command = sample_command(init_replay_buffer, last_few)
            writer.add_scalar('Command desired reward/Episode', command[0], i_episode)    # write loss to a graph
            writer.add_scalar('Command horizon/Episode', command[1], i_episode)    # write loss to a graph
            observation = env.reset()
            episode_mem = {'observation':[],
                           'action':[],
                           'reward':[],}
            done=False
            while not done:
                action = policy(torch.tensor([observation]).double(), torch.tensor([command]).double())
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                
                observation=new_observation
                command[0]-= reward
                command[1] = max(1, command[1]-1)
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            replay_buffer.append(episode_mem)
            i_episode+=1
            if log_to_tensorboard: writer.add_scalar('Return/Episode', sum(episode_mem['reward']), i_episode)    # write loss to a graph
            print("Episode {} finished after {} timesteps. Return = {}".format(i_episode, len(episode_mem['observation']), sum(episode_mem['reward'])))
        env.close()
    except KeyboardInterrupt:
        env.close()
    replay_buffer = sorted(replay_buffer, key=lambda x:x['return'])[-replay_size:]
    return replay_buffer

def sample_command(replay_buffer, last_few):
    if len(replay_buffer)==0:
        return [1, 1]
    else:
        command_samples = replay_buffer[-last_few:]
        lengths = [mem['episode_len'] for mem in command_samples]
        returns = [mem['return'] for mem in command_samples]
        mean_return, std_return = np.mean(returns), np.std(returns)
        command_horizon = np.mean(lengths)
        desired_reward = np.random.uniform(mean_return, mean_return+std_return)
        return [desired_reward, command_horizon]

In [24]:
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64, log_to_tensorboard=True):
    global i_updates
    all_costs = []
    for i in range(n_updates):
        batch_observations = np.zeros((batch_size, np.prod(env.observation_space.shape)))
        batch_commands = np.zeros((batch_size, 2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_t1 = np.random.randint(0, len(replay_buffer[sample_episode]['observation']))
            sample_t2 = len(replay_buffer[sample_episode]['observation'])
            ##sample_t2 = np.random.randint(sample_t1+1, len(replay_buffer[sample_episode]['observation'])+1)
            sample_horizon = sample_t2-sample_t1
            sample_mem = replay_buffer[sample_episode]['observation'][sample_t1]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_t1:sample_t2])
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_t1]
            batch_observations[b] = sample_mem
            batch_commands[b] = [sample_desired_reward, sample_horizon]
            batch_label[b] = label
        batch_observations = torch.tensor(batch_observations).double()
        batch_commands = torch.tensor(batch_commands).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_observations, batch_commands)
        cost = F.cross_entropy(pred, batch_label)
        if log_to_tensorboard: writer.add_scalar('Cost/NN update', cost.item() , i_updates)    # write loss to a graph
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
        i_updates+=1
    return np.mean(all_costs)

In [25]:
def create_greedy_policy(policy_network):
    def policy(obs, command):
        action_logits = policy_network(obs, command)
        action = np.argmax(action_logits.detach().numpy())
        return action
    return policy

def create_stochastic_policy(policy_network):
    def policy(obs, command):
        action_logits = policy_network(obs, command)
        action_probs = F.softmax(action_logits, dim=-1)
        action = torch.distributions.Categorical(action_probs).sample().item()
        return action
    return policy

In [30]:
i_episode=0
i_updates=0 #number of parameter updates to the neural network
replay_buffer = []
log_to_tensorboard = True

replay_size = 600
last_few = 50
batch_size = 128
n_warm_up_episodes = 50
n_episodes_per_iter = 50
n_updates_per_iter = 100
command_scale = 0.02
lr = 0.001

agent = FCNN_AGENT(command_scale).double()
agent.create_optimizer(lr)

stochastic_policy = create_stochastic_policy(agent)
greedy_policy = create_greedy_policy(agent)

In [31]:
# SET UP TRAINING VISUALISATION
# SET UP TRAINING VISUALISATION
if log_to_tensorboard: from torch.utils.tensorboard import SummaryWriter
if log_to_tensorboard: writer = SummaryWriter() # we will use this to show our models performance on a graph using tensorboard

In [32]:
#Collect warm up episodes
replay_buffer = collect_experience(random_policy, replay_buffer, replay_size, last_few, n_warm_up_episodes, log_to_tensorboard)
train_net(agent, replay_buffer, n_updates_per_iter, batch_size, log_to_tensorboard)

Episode 1 finished after 113 timesteps. Return = -390.6940084980434
Episode 2 finished after 95 timesteps. Return = -120.41847788549137
Episode 3 finished after 83 timesteps. Return = -63.466702803711684
Episode 4 finished after 92 timesteps. Return = -483.2119955776536
Episode 5 finished after 76 timesteps. Return = -65.54904155474622
Episode 6 finished after 85 timesteps. Return = -330.4744824335155
Episode 7 finished after 71 timesteps. Return = -101.51475416269122
Episode 8 finished after 132 timesteps. Return = -106.36861748934871
Episode 9 finished after 80 timesteps. Return = -139.41218679279342
Episode 10 finished after 90 timesteps. Return = -118.64424067999293
Episode 11 finished after 122 timesteps. Return = -123.55016476157775
Episode 12 finished after 77 timesteps. Return = -129.0480454451287
Episode 13 finished after 76 timesteps. Return = -159.92647263149016
Episode 14 finished after 137 timesteps. Return = -358.05057496315317
Episode 15 finished after 62 timesteps. Retu

1.3842409914766693

In [33]:
n_iters = 1000
for i in range(n_iters):
    replay_buffer = collect_experience(stochastic_policy, replay_buffer, replay_size, last_few, n_episodes_per_iter, log_to_tensorboard)
    train_net(agent, replay_buffer, n_updates_per_iter, batch_size, log_to_tensorboard)

Episode 51 finished after 89 timesteps. Return = -382.6634132320046
Episode 52 finished after 66 timesteps. Return = -96.67326599693764
Episode 53 finished after 83 timesteps. Return = -90.11398104032524
Episode 54 finished after 101 timesteps. Return = -240.75159988453566
Episode 55 finished after 95 timesteps. Return = -428.78147663886165
Episode 56 finished after 70 timesteps. Return = -119.74727539795562
Episode 57 finished after 62 timesteps. Return = -63.50740351740876
Episode 58 finished after 90 timesteps. Return = -113.2061045242965
Episode 59 finished after 60 timesteps. Return = -64.30061205143289
Episode 60 finished after 144 timesteps. Return = -245.41588137987995
Episode 61 finished after 75 timesteps. Return = -260.00580021195094
Episode 62 finished after 90 timesteps. Return = -372.659845652631
Episode 63 finished after 79 timesteps. Return = -96.01097819183681
Episode 64 finished after 90 timesteps. Return = -238.6632391503006
Episode 65 finished after 66 timesteps. Re

Episode 173 finished after 113 timesteps. Return = -146.16898015623693
Episode 174 finished after 111 timesteps. Return = -489.8619186321915
Episode 175 finished after 154 timesteps. Return = -275.6482248869564
Episode 176 finished after 89 timesteps. Return = -224.8834971080876
Episode 177 finished after 63 timesteps. Return = -71.52222855604283
Episode 178 finished after 82 timesteps. Return = -140.30410781962425
Episode 179 finished after 79 timesteps. Return = -122.02366125992998
Episode 180 finished after 124 timesteps. Return = -165.93310409176826
Episode 181 finished after 103 timesteps. Return = -276.1198114251433
Episode 182 finished after 99 timesteps. Return = -130.49951256522752
Episode 183 finished after 99 timesteps. Return = -265.4959204137937
Episode 184 finished after 73 timesteps. Return = -149.19292178189835
Episode 185 finished after 109 timesteps. Return = -105.72955728693329
Episode 186 finished after 118 timesteps. Return = -315.0724182603636
Episode 187 finished

Episode 293 finished after 124 timesteps. Return = -122.74732759471013
Episode 294 finished after 112 timesteps. Return = -303.41056760790195
Episode 295 finished after 100 timesteps. Return = -125.1671671613999
Episode 296 finished after 105 timesteps. Return = -483.78897310718884
Episode 297 finished after 78 timesteps. Return = -179.34726002064474
Episode 298 finished after 113 timesteps. Return = -132.6269211254701
Episode 299 finished after 102 timesteps. Return = -122.54722847844738
Episode 300 finished after 91 timesteps. Return = -113.98932139236821
Episode 301 finished after 106 timesteps. Return = -163.19702207224162
Episode 302 finished after 117 timesteps. Return = -101.54200341571133
Episode 303 finished after 101 timesteps. Return = -364.2796980802363
Episode 304 finished after 96 timesteps. Return = -92.37097074650198
Episode 305 finished after 113 timesteps. Return = -122.6096648109436
Episode 306 finished after 79 timesteps. Return = -116.2119978044935
Episode 307 fini

Episode 413 finished after 66 timesteps. Return = -171.60735951303937
Episode 414 finished after 116 timesteps. Return = -308.62132575207926
Episode 415 finished after 102 timesteps. Return = -46.12720090508134
Episode 416 finished after 68 timesteps. Return = -227.60044391782316
Episode 417 finished after 69 timesteps. Return = -66.6542079852322
Episode 418 finished after 58 timesteps. Return = -120.95926202800892
Episode 419 finished after 93 timesteps. Return = -121.97172074324645
Episode 420 finished after 83 timesteps. Return = -112.05950002555619
Episode 421 finished after 77 timesteps. Return = -61.81662354310717
Episode 422 finished after 93 timesteps. Return = -198.84863964387483
Episode 423 finished after 88 timesteps. Return = -85.31832746328156
Episode 424 finished after 100 timesteps. Return = -82.797372410277
Episode 425 finished after 111 timesteps. Return = -153.3357664389659
Episode 426 finished after 103 timesteps. Return = -150.95175225912004
Episode 427 finished aft

Episode 536 finished after 81 timesteps. Return = -141.82492359367524
Episode 537 finished after 68 timesteps. Return = -85.90298487671994
Episode 538 finished after 127 timesteps. Return = -102.10353714031793
Episode 539 finished after 61 timesteps. Return = -118.91272534123863
Episode 540 finished after 71 timesteps. Return = -121.16697081074602
Episode 541 finished after 138 timesteps. Return = -140.38418268276484
Episode 542 finished after 134 timesteps. Return = -341.0148507021874
Episode 543 finished after 86 timesteps. Return = -132.48753123742733
Episode 544 finished after 107 timesteps. Return = -134.97071038142502
Episode 545 finished after 56 timesteps. Return = -93.0747473823195
Episode 546 finished after 61 timesteps. Return = -62.25213987671873
Episode 547 finished after 67 timesteps. Return = -62.296819692297284
Episode 548 finished after 123 timesteps. Return = -135.9170623854852
Episode 549 finished after 119 timesteps. Return = -122.44812014021304
Episode 550 finished

...

Episode 10733 finished after 328 timesteps. Return = -183.57168888177472
Episode 10734 finished after 138 timesteps. Return = -37.39602905894847
Episode 10735 finished after 113 timesteps. Return = -177.94119679084088
Episode 10736 finished after 77 timesteps. Return = -98.92459300290372
Episode 10737 finished after 86 timesteps. Return = -106.29376374524459
Episode 10738 finished after 383 timesteps. Return = -271.67976294423346
Episode 10739 finished after 100 timesteps. Return = -101.72536884326921
Episode 10740 finished after 311 timesteps. Return = -119.64753355685075
Episode 10741 finished after 190 timesteps. Return = -80.3368062784015
Episode 10742 finished after 157 timesteps. Return = -106.76720062733303
Episode 10743 finished after 309 timesteps. Return = -135.50009760632528
Episode 10744 finished after 239 timesteps. Return = -108.38732219124074
Episode 10745 finished after 278 timesteps. Return = -163.4393946669654
Episode 10746 finished after 170 timesteps. Return = -96.0

Episode 10850 finished after 580 timesteps. Return = -243.84958453780087
Episode 10851 finished after 181 timesteps. Return = -142.11493040144745
Episode 10852 finished after 268 timesteps. Return = -121.68796835945638
Episode 10853 finished after 324 timesteps. Return = -257.49141431020155
Episode 10854 finished after 146 timesteps. Return = -267.6956864012601
Episode 10855 finished after 536 timesteps. Return = -334.7815754531735
Episode 10856 finished after 189 timesteps. Return = -232.17162131012103
Episode 10857 finished after 409 timesteps. Return = -134.69590538478664
Episode 10858 finished after 350 timesteps. Return = -93.02210942730522
Episode 10859 finished after 290 timesteps. Return = -287.51864338538485
Episode 10860 finished after 393 timesteps. Return = -277.06819723553843
Episode 10861 finished after 162 timesteps. Return = -12.251041983652883
Episode 10862 finished after 71 timesteps. Return = -94.60622443674157
Episode 10863 finished after 112 timesteps. Return = -20

Episode 10965 finished after 307 timesteps. Return = -268.7664417969695
Episode 10966 finished after 242 timesteps. Return = -248.78221174666538
Episode 10967 finished after 111 timesteps. Return = -77.57929020844722
Episode 10968 finished after 178 timesteps. Return = -100.9216698284497
Episode 10969 finished after 94 timesteps. Return = -43.56589487894858
Episode 10970 finished after 319 timesteps. Return = -256.2323374889502
Episode 10971 finished after 105 timesteps. Return = -243.21508900866888
Episode 10972 finished after 83 timesteps. Return = -106.19558179350419
Episode 10973 finished after 411 timesteps. Return = -50.0485559615093
Episode 10974 finished after 264 timesteps. Return = -104.40196384894145
Episode 10975 finished after 78 timesteps. Return = -171.57284795648957
Episode 10976 finished after 356 timesteps. Return = -85.37605588882722
Episode 10977 finished after 75 timesteps. Return = -109.9953660724099
Episode 10978 finished after 137 timesteps. Return = -196.077360

Episode 11080 finished after 216 timesteps. Return = -138.0907612385591
Episode 11081 finished after 151 timesteps. Return = -1.9515855874326604
Episode 11082 finished after 236 timesteps. Return = -165.3168693255198
Episode 11083 finished after 91 timesteps. Return = -174.91662442252368
Episode 11084 finished after 227 timesteps. Return = -68.87648271183699
Episode 11085 finished after 496 timesteps. Return = -219.57806116786645
Episode 11086 finished after 85 timesteps. Return = -106.561020582802
Episode 11087 finished after 770 timesteps. Return = -317.7336018934359
Episode 11088 finished after 240 timesteps. Return = -228.40717649093443
Episode 11089 finished after 252 timesteps. Return = -87.4715262817219
Episode 11090 finished after 437 timesteps. Return = -277.3439472888855
Episode 11091 finished after 183 timesteps. Return = 15.000753237321561
Episode 11092 finished after 139 timesteps. Return = -12.958991922514187
Episode 11093 finished after 193 timesteps. Return = -70.053649

KeyboardInterrupt: 

In [165]:
#torch.save(agent.state_dict(), 'checkpoints/lunar_lander_64x64_checkpoint_0.pt')
#agent.load_state_dict(torch.load('checkpoints/lunar_lander_32x32_checkpoint_0.pt'))

In [74]:
visualise_agent(greedy_policy, command=[150, 400], n=5)

Episode 0 finished after 156 timesteps. Return = -241.49785131822478
Episode 1 finished after 297 timesteps. Return = -358.99471816394623
Episode 2 finished after 211 timesteps. Return = -314.8731776425535
Episode 3 finished after 102 timesteps. Return = -224.18053479219859
Episode 4 finished after 412 timesteps. Return = -387.7649308825705


In [73]:
visualise_agent(stochastic_policy, command=[150, 400], n=5)

Episode 0 finished after 101 timesteps. Return = -131.8027246567254


In [25]:
print([mem['return'] for mem in replay_buffer])

[-63.70254888830934, -63.638939025122625, -63.5994898557033, -63.49465027645706, -63.46673824051337, -63.36871712910339, -63.3170788909209, -63.268455814456246, -63.26158837333688, -63.22068659038845, -63.21192519962132, -63.159962894358316, -63.035593454982134, -63.035425263206434, -62.97096940697155, -62.943593599875086, -62.92596251667162, -62.91281822394709, -62.90096774806254, -62.89021254680336, -62.88432112394105, -62.833528924538854, -62.77959557612017, -62.667310065127865, -62.66679486512233, -62.629278971083366, -62.61750379938199, -62.575461411676095, -62.47476364144245, -62.2956868138306, -62.28361089060018, -62.21614293189815, -62.211379210167564, -62.19752032429969, -62.17571584274293, -62.118940774335755, -62.08795198808494, -61.997331174933244, -61.92102486534531, -61.8701775567973, -61.79702597554188, -61.78349969009108, -61.761231874595126, -61.75852742540508, -61.71683109542317, -61.6446895161059, -61.506866501602374, -61.489387372573006, -61.48727633614841, -61.4803

# Previous Code

In [ ]:
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64):
    all_costs = []
    for i in range(n_updates):
        batch_input = np.zeros((batch_size, np.prod(env.observation_space.shape)+2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_horizon = np.random.randint(1, len(replay_buffer[sample_episode]['observation'])+1)
            sample_mem_idx = np.random.randint(0, len(replay_buffer[sample_episode]['observation'])+1-sample_horizon)
            sample_mem = replay_buffer[sample_episode]['observation'][sample_mem_idx]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_mem_idx:sample_mem_idx+sample_horizon])
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_mem_idx]
            batch_input[b] = network_input
            batch_label[b] = label
        batch_input = torch.tensor(batch_input).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_input)
        cost = F.cross_entropy(pred, batch_label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)

In [8]:
def train_net(policy_net, episode_mem, n_samples = 5): #stochastic gradient descent
    all_costs = []
    for i in range(n_samples):
        sample_horizon = np.random.randint(1, len(episode_mem['observation'])+1)
        sample_mem_idx = np.random.randint(0, len(episode_mem['observation'])+1-sample_horizon)
        sample_mem = episode_mem['observation'][sample_mem_idx]
        sample_desired_reward = sum(episode_mem['reward'][sample_mem_idx:sample_mem_idx+sample_horizon])
        network_input = torch.tensor(np.append(sample_mem, [sample_desired_reward, sample_horizon])).double()
        label = torch.tensor([episode_mem['action'][sample_mem_idx]]).double()
        
        pred = policy_net(network_input)
        cost = F.binary_cross_entropy(pred, label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)
    

In [33]:
def train(policy_net, n_episodes=100):
    global i_episode
    global epsilon
    try:
        for _ in range(n_episodes):
            observation = env.reset()
            episode_mem = {'observation':[],
                            'action':[],
                            'reward':[],
                            'done':[]}
            done=False
            while not done:
                network_input = torch.tensor(np.append(observation, [desired_reward, command_horizon])).double()
                action_prob = policy_net(network_input)
                action = np.random.binomial(1, action_prob.item())
                #action = int(action_prob.item()>0.5)
                if np.random.rand()<epsilon: action = np.random.randint(0, 2)
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                episode_mem['done'].append(done)
                
                observation=new_observation
                epsilon*=0.999
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            mean_cost = train_net(policy_net, episode_mem)
            
            i_episode+=1
            print("Episode {} finished after {} timesteps. Epsilon={} Mean Cost={}".format(i_episode, len(episode_mem['observation']), epsilon, mean_cost))
        env.close()
    except KeyboardInterrupt:
        env.close()

## Previous version

In [55]:
import gym
import time
import torch
import numpy as np
from copy import deepcopy
import torch.nn.functional as F

def random_policy(obs):
    return np.random.randint(env.action_space.n)

#Visualise agent function
def visualise_agent(policy, command, command_scale, n=5):
    try:
        for trial_i in range(n):
            current_command = deepcopy(command)
            observation = env.reset()
            done=False
            t=0
            episode_return=0
            while not done:
                env.render()
                network_input = torch.tensor(np.append(observation, np.array(current_command)*command_scale)).double()
                policy_action = policy(network_input)
                observation, reward, done, info = env.step(policy_action)
                episode_return+=reward
                #time.sleep(0.1)
                current_command[0]-= reward
                current_command[1] = max(1, current_command[1]-1)
                
                t+=1
            env.render()
            time.sleep(1.5)
            print("Episode {} finished after {} timesteps. Return = {}".format(trial_i, t, episode_return))
        env.close()
    except KeyboardInterrupt:
        env.close()
        
#Behaviour function - Neural Network
class FCNN_AGENT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(np.prod(env.observation_space.shape)+2, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, env.action_space.n)
        )
    
    def forward(self, x):
        x = self.fc(x)
        return x
    
    def create_optimizer(self, lr):
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

#Full the replay buffer with more experience
def collect_experience(policy, replay_buffer, replay_size, last_few, n_episodes=100, command_scale=0.01, log_to_tensorboard=True):
    global i_episode
    init_replay_buffer = deepcopy(replay_buffer)
    try:
        for _ in range(n_episodes):
            command = sample_command(init_replay_buffer, last_few)
            if log_to_tensorboard: writer.add_scalar('Command desired reward/Episode', command[0], i_episode)    # write loss to a graph
            if log_to_tensorboard: writer.add_scalar('Command horizon/Episode', command[1], i_episode)    # write loss to a graph
            observation = env.reset()
            episode_mem = {'observation':[],
                            'action':[],
                            'reward':[]}
            done=False
            while not done:
                network_input = torch.tensor(np.append(observation, np.array(command)*command_scale)).double()
                action = policy(network_input)
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                
                observation=new_observation
                #command[0]-= reward
                command[0] = max(1, command[0]-reward)
                command[1] = max(1, command[1]-1)
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            replay_buffer.append(episode_mem)
            i_episode+=1
            if log_to_tensorboard: writer.add_scalar('Return/Episode', sum(episode_mem['reward']), i_episode)    # write loss to a graph
            print("Episode {} finished after {} timesteps. Return = {}".format(i_episode, len(episode_mem['observation']), sum(episode_mem['reward'])))
        env.close()
    except KeyboardInterrupt:
        env.close()
    replay_buffer = sorted(replay_buffer, key=lambda x:x['return'])[-replay_size:]
    return replay_buffer

#Sample exploratory command
def sample_command(replay_buffer, last_few):
    if len(replay_buffer)==0:
        return [1, 1]
    else:
        command_samples = replay_buffer[-last_few:]
        lengths = [mem['episode_len'] for mem in command_samples]
        returns = [mem['return'] for mem in command_samples]
        mean_return, std_return = np.mean(returns), np.std(returns)
        command_horizon = np.mean(lengths)
        desired_reward = np.random.uniform(mean_return, mean_return+std_return)
        return [desired_reward, command_horizon]

#Improve behviour function by training on replay buffer
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64, command_scale=0.01):
    global i_updates
    all_costs = []
    for i in range(n_updates):
        batch_input = np.zeros((batch_size, np.prod(env.observation_space.shape)+2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_t1 = np.random.randint(0, len(replay_buffer[sample_episode]['observation']))
            sample_t2 = len(replay_buffer[sample_episode]['observation'])
            sample_horizon = (sample_t2-sample_t1) * command_scale
            sample_mem = replay_buffer[sample_episode]['observation'][sample_t1]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_t1:sample_t2])*command_scale
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_t1]
            batch_input[b] = network_input
            batch_label[b] = label
        batch_input = torch.tensor(batch_input).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_input)
        cost = F.cross_entropy(pred, batch_label)
        if log_to_tensorboard: writer.add_scalar('Cost/NN update', cost.item() , i_updates)    # write loss to a graph
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
        i_updates+=1
    return np.mean(all_costs)

#Return a greedy policy from a given network
def create_greedy_policy(policy_network):
    def policy(obs):
        action_logits = policy_network(obs)
        action = np.argmax(action_logits.detach().numpy())
        return action
    return policy

#Return a stochastic policy from a given network
def create_stochastic_policy(policy_network):
    def policy(obs):
        action_logits = policy_network(obs)
        action_probs = F.softmax(action_logits, dim=-1)
        action = torch.distributions.Categorical(action_probs).sample().item()
        return action
    return policy

In [56]:
#Define hyperparameters, initialize behaviour function
env = gym.make('LunarLander-v2')
i_episode=0
i_updates=0
replay_buffer = []
log_to_tensorboard = True 

replay_size = 600
last_few = 75
batch_size = 32
n_warm_up_episodes = 50
n_episodes_per_iter = 50
n_updates_per_iter = 300
command_scale = 0.01
lr = 0.001

agent = FCNN_AGENT().double()
agent.create_optimizer(lr)

stochastic_policy = create_stochastic_policy(agent)
greedy_policy = create_greedy_policy(agent)

# SET UP TRAINING VISUALISATION
if log_to_tensorboard: from torch.utils.tensorboard import SummaryWriter
if log_to_tensorboard: writer = SummaryWriter() # we will use this to show our models performance on a graph using tensorboard

#Collect warm up episodes
replay_buffer = collect_experience(random_policy, replay_buffer, replay_size, last_few, n_warm_up_episodes, command_scale, log_to_tensorboard)
train_net(agent, replay_buffer, n_updates_per_iter, batch_size, command_scale)

Episode 1 finished after 78 timesteps. Return = -128.4480682875527
Episode 2 finished after 89 timesteps. Return = -387.88715583374216
Episode 3 finished after 77 timesteps. Return = -95.10879230530807
Episode 4 finished after 72 timesteps. Return = -102.74438279096503
Episode 5 finished after 116 timesteps. Return = -253.90123618001869
Episode 6 finished after 107 timesteps. Return = -178.8621093404991
Episode 7 finished after 110 timesteps. Return = -424.2903105389424
Episode 8 finished after 69 timesteps. Return = -90.45928541886127
Episode 9 finished after 85 timesteps. Return = -90.34062619706825
Episode 10 finished after 110 timesteps. Return = -257.37052024828347
Episode 11 finished after 95 timesteps. Return = -171.70578091801366
Episode 12 finished after 65 timesteps. Return = -264.4237434177501
Episode 13 finished after 128 timesteps. Return = -146.38984817020642
Episode 14 finished after 73 timesteps. Return = -95.87199679628412
Episode 15 finished after 67 timesteps. Return

1.3918637607680855

In [57]:
#Collect experience and train behaviour function for given number of iterations
n_iters = 1000
for i in range(n_iters):
    replay_buffer = collect_experience(stochastic_policy, replay_buffer, replay_size, last_few, n_episodes_per_iter, command_scale, log_to_tensorboard)
    train_net(agent, replay_buffer, n_updates_per_iter, batch_size, command_scale)

Episode 51 finished after 137 timesteps. Return = -446.3506995863737
Episode 52 finished after 77 timesteps. Return = -140.5196864731071
Episode 53 finished after 70 timesteps. Return = -85.56728471718694
Episode 54 finished after 71 timesteps. Return = -94.78256978743447
Episode 55 finished after 114 timesteps. Return = -65.1801625049948
Episode 56 finished after 71 timesteps. Return = -315.67881711784116
Episode 57 finished after 120 timesteps. Return = -67.65283374333278
Episode 58 finished after 92 timesteps. Return = -413.1242067143709
Episode 59 finished after 96 timesteps. Return = -134.2708548742756
Episode 60 finished after 96 timesteps. Return = -115.75873117970687
Episode 61 finished after 88 timesteps. Return = -127.17007197006436
Episode 62 finished after 112 timesteps. Return = -301.4526746821167
Episode 63 finished after 113 timesteps. Return = -211.53988109680003
Episode 64 finished after 68 timesteps. Return = -119.3757190542396
Episode 65 finished after 93 timesteps. 

Episode 174 finished after 69 timesteps. Return = -104.16899601506384
Episode 175 finished after 84 timesteps. Return = -96.07286939569624
Episode 176 finished after 131 timesteps. Return = -139.4199800131397
Episode 177 finished after 74 timesteps. Return = -91.46291264096092
Episode 178 finished after 115 timesteps. Return = -90.51635088079755
Episode 179 finished after 98 timesteps. Return = -292.12224853522514
Episode 180 finished after 79 timesteps. Return = -143.27721094359757
Episode 181 finished after 63 timesteps. Return = -124.11629519864661
Episode 182 finished after 84 timesteps. Return = -133.22669053051544
Episode 183 finished after 95 timesteps. Return = -171.2783776465923
Episode 184 finished after 96 timesteps. Return = -93.28849817463086
Episode 185 finished after 92 timesteps. Return = -114.1557484370562
Episode 186 finished after 62 timesteps. Return = -102.55848158584956
Episode 187 finished after 121 timesteps. Return = -151.4838767167584
Episode 188 finished afte

Episode 293 finished after 98 timesteps. Return = -126.0177373450193
Episode 294 finished after 85 timesteps. Return = -41.33706913169587
Episode 295 finished after 110 timesteps. Return = -104.89408300251816
Episode 296 finished after 122 timesteps. Return = -122.06720661409226
Episode 297 finished after 67 timesteps. Return = -93.27479221165001
Episode 298 finished after 92 timesteps. Return = -111.51403266195848
Episode 299 finished after 96 timesteps. Return = -278.2307061737931
Episode 300 finished after 59 timesteps. Return = -68.63417569381657
Episode 301 finished after 113 timesteps. Return = -174.5064029495523
Episode 302 finished after 108 timesteps. Return = -138.43405525667737
Episode 303 finished after 87 timesteps. Return = -364.50838279260915
Episode 304 finished after 122 timesteps. Return = -90.0417980348177
Episode 305 finished after 63 timesteps. Return = -74.82944791207059
Episode 306 finished after 78 timesteps. Return = 12.349499040470633
Episode 307 finished afte

Episode 412 finished after 107 timesteps. Return = -190.50868161911495
Episode 413 finished after 89 timesteps. Return = -91.3439183969028
Episode 414 finished after 65 timesteps. Return = -95.84527173026802
Episode 415 finished after 106 timesteps. Return = -191.69772948261556
Episode 416 finished after 64 timesteps. Return = -120.5916127759939
Episode 417 finished after 69 timesteps. Return = -99.47554883556316
Episode 418 finished after 69 timesteps. Return = -138.91836274515907
Episode 419 finished after 64 timesteps. Return = -117.55656377805636
Episode 420 finished after 112 timesteps. Return = -100.63873253717723
Episode 421 finished after 73 timesteps. Return = -171.74789328967256
Episode 422 finished after 88 timesteps. Return = -80.93309778150977
Episode 423 finished after 95 timesteps. Return = -227.75923425296233
Episode 424 finished after 72 timesteps. Return = -122.62037874926172
Episode 425 finished after 61 timesteps. Return = -145.92015232939673
Episode 426 finished af

Episode 531 finished after 61 timesteps. Return = -246.37178230884416
Episode 532 finished after 69 timesteps. Return = -184.35148436013293
Episode 533 finished after 88 timesteps. Return = -162.70473431205681
Episode 534 finished after 71 timesteps. Return = -110.32276938173662
Episode 535 finished after 89 timesteps. Return = -191.55526324541177
Episode 536 finished after 88 timesteps. Return = -137.48470208927156
Episode 537 finished after 104 timesteps. Return = -347.35429462174017
Episode 538 finished after 83 timesteps. Return = -128.69897913425598
Episode 539 finished after 101 timesteps. Return = -321.2059091532176
Episode 540 finished after 112 timesteps. Return = -138.7140622015092
Episode 541 finished after 100 timesteps. Return = -186.51693224204348
Episode 542 finished after 60 timesteps. Return = -83.74087998108158
Episode 543 finished after 91 timesteps. Return = -271.50812190237775
Episode 544 finished after 92 timesteps. Return = -82.63524718879307
Episode 545 finished

...

Episode 10101 finished after 78 timesteps. Return = -253.9123297690235
Episode 10102 finished after 98 timesteps. Return = -108.68588536881725
Episode 10103 finished after 109 timesteps. Return = -276.3130128492782
Episode 10104 finished after 73 timesteps. Return = -137.2722507679424
Episode 10105 finished after 74 timesteps. Return = -209.2157125644
Episode 10106 finished after 86 timesteps. Return = -88.7444603670404
Episode 10107 finished after 77 timesteps. Return = -356.52679714941866
Episode 10108 finished after 138 timesteps. Return = -220.45139805186864
Episode 10109 finished after 107 timesteps. Return = -79.68661487775253
Episode 10110 finished after 121 timesteps. Return = -276.6277944031742
Episode 10111 finished after 90 timesteps. Return = -130.78561267113594
Episode 10112 finished after 67 timesteps. Return = -98.1422718651761
Episode 10113 finished after 64 timesteps. Return = -158.65167193325232
Episode 10114 finished after 72 timesteps. Return = -217.62920000370764
E

Episode 10217 finished after 103 timesteps. Return = -225.2060862938422
Episode 10218 finished after 116 timesteps. Return = -223.32965744676562
Episode 10219 finished after 73 timesteps. Return = -178.30485527539557
Episode 10220 finished after 68 timesteps. Return = -173.2401215020153
Episode 10221 finished after 102 timesteps. Return = -193.70920019630262
Episode 10222 finished after 96 timesteps. Return = -250.47192846547958
Episode 10223 finished after 85 timesteps. Return = -274.8602768315085
Episode 10224 finished after 64 timesteps. Return = -210.3944776394335
Episode 10225 finished after 93 timesteps. Return = -432.00716798769065
Episode 10226 finished after 106 timesteps. Return = -261.9569655213727
Episode 10227 finished after 120 timesteps. Return = -204.1786505505013
Episode 10228 finished after 123 timesteps. Return = -222.61933816956827
Episode 10229 finished after 102 timesteps. Return = -196.49482821373385
Episode 10230 finished after 78 timesteps. Return = -267.295428

Episode 10333 finished after 113 timesteps. Return = -260.2838687874207
Episode 10334 finished after 110 timesteps. Return = -235.0067838682809
Episode 10335 finished after 77 timesteps. Return = -96.25937692869358
Episode 10336 finished after 102 timesteps. Return = -238.38710736346502
Episode 10337 finished after 74 timesteps. Return = -223.8691308182398
Episode 10338 finished after 75 timesteps. Return = -226.56699931855235
Episode 10339 finished after 147 timesteps. Return = -243.79549509325182
Episode 10340 finished after 110 timesteps. Return = 30.37227483745673
Episode 10341 finished after 105 timesteps. Return = -119.33070727404453
Episode 10342 finished after 73 timesteps. Return = -253.1118530820678
Episode 10343 finished after 112 timesteps. Return = -173.84977284027468
Episode 10344 finished after 107 timesteps. Return = -149.31891424307548
Episode 10345 finished after 82 timesteps. Return = -231.3517291713811
Episode 10346 finished after 90 timesteps. Return = -287.0110114

Episode 10448 finished after 115 timesteps. Return = -289.59448611566364
Episode 10449 finished after 70 timesteps. Return = -85.0903009184932
Episode 10450 finished after 64 timesteps. Return = -115.96613238408956
Episode 10451 finished after 108 timesteps. Return = -176.49511329948885
Episode 10452 finished after 123 timesteps. Return = -171.17851497414946
Episode 10453 finished after 143 timesteps. Return = -123.82929111347866
Episode 10454 finished after 86 timesteps. Return = -205.61433903770478
Episode 10455 finished after 82 timesteps. Return = -193.132453877897
Episode 10456 finished after 76 timesteps. Return = -140.6871474685144
Episode 10457 finished after 93 timesteps. Return = -150.9912237001328
Episode 10458 finished after 127 timesteps. Return = -37.23500982957983
Episode 10459 finished after 75 timesteps. Return = -85.85557685041854
Episode 10460 finished after 73 timesteps. Return = -221.7324207450483
Episode 10461 finished after 108 timesteps. Return = -107.3159184794

KeyboardInterrupt: 

In [51]:
#Visualise final trained agent
visualise_agent(greedy_policy, command=[400, 400], command_scale=command_scale, n=3)

Episode 0 finished after 125 timesteps. Return = 125.0
